In [1]:
MONGO_URI = "Use env or Direct path for your Mongo URI"

In [6]:
DB_NAME = 'Name of your databse'
COLLECTION_NAME = 'Name of your collection'
INDEX_NAME = 'Name of your index'

In [7]:
from pymongo import MongoClient

In [8]:
client = MongoClient(MONGO_URI)
print ("Atlas client initialized") 

Atlas client initialized


In [9]:

MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME] 

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [1]:
embeddings = download_hugging_face_embeddings()

In [13]:
from langchain_community.vectorstores import MongoDBAtlasVectorSearch

In [14]:
# vector_search = MongoDBAtlasVectorSearch.from_documents(
#     documents=docs,
#     embedding=embeddings,
#     collection=MONGODB_COLLECTION,
#     index_name=INDEX_NAME
# )

#data have been embedded on mongoDB, initially populating collection

In [2]:
vector_search_again = MongoDBAtlasVectorSearch(
    collection=MONGODB_COLLECTION,
    embedding=embeddings
)

#querying from existing collection

In [16]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
 
Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [17]:
from langchain.prompts import PromptTemplate 

In [18]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT} 

In [19]:
from langchain.llms import CTransformers

In [21]:
llm=CTransformers(model="path to your model",
                  model_type="llama",
                  config={'max_new_tokens':512,    
                          'temperature':0.8})

In [3]:
from langchain.chains import RetrievalQA 

In [10]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = vector_search_again.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT} 
)

In [6]:
question = "What is the cause of allergies" 

In [8]:
result = qa.run(question)